# Tensorflow MNIST Feature Squeezing Defense demo

>⚠️ **Warning:** Some of the attacks in this demo, _deepfool_ and _CW_ in particular, are computationally expensive and will take a very long to complete if run using the CPUs found in a typical personal computer.
> For this reason, it is highly recommended that you run these demos on a CUDA-compatible GPU.

This notebook contains a demonstration of how to use Dioptra to run experiments that investigate the effectiveness of the feature-squeezing defense against a series of evasion attacks against a neural network model.

## Setup

Below we import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected,

In [ ]:
# Import packages from the Python standard library
import os
import pprint
import time
import warnings
from pathlib import Path
from typing import Tuple

# Filter out warning messages
warnings.filterwarnings("ignore")

# Address for connecting the docker container to exposed ports on the host device
HOST_DOCKER_INTERNAL = "host.docker.internal"
# HOST_DOCKER_INTERNAL = "172.17.0.1"

# Dioptra API ports
RESTAPI_PORT = "30080"
MLFLOW_TRACKING_PORT = "35000"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = (
    f"http://{HOST_DOCKER_INTERNAL}:{RESTAPI_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{RESTAPI_PORT}"
)

# Override the DIOPTRA_RESTAPI_URI variable, used to connect to RESTful API service
os.environ["DIOPTRA_RESTAPI_URI"] = RESTAPI_ADDRESS

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = (
    f"http://{HOST_DOCKER_INTERNAL}:{MLFLOW_TRACKING_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{MLFLOW_TRACKING_PORT}"
)

# Override the MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Base API address
RESTAPI_API_BASE = f"{RESTAPI_ADDRESS}/api"

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Import third-party Python packages
import numpy as np
import requests
from mlflow.tracking import MlflowClient

# Import utils.py file
import utils

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

# Configure paths to datasets
data_path_mnist = "/nfs/data/Mnist"
data_path_imagenet = "/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-1000"

# Configure experiment namespaces
mlflow_queue = "tensorflow_gpu" # Change this queue if needed.
uid = "plugin".lower() #Replace placeholder with your desired UserID. This can be anything, but will be viewable to other users of the platform.
experiment = uid + "_feature_squeeze"
model_id = "1" #change this if you have multiple models with the same name. Defaults to str(1)
mnist_model = experiment + "_le_net" 
mnist_shallow = experiment + "_shallow_net"
mnist_logit = mnist_model + "_logit"
imagenet_model = experiment + "_default_pretrained_mobilenet_logits/" 

## Dataset

We obtained a copy of the MNIST dataset as part of the startup process invoked by `make demo`.
The training and testing images for the MNIST dataset are stored within the `data/` directory as PNG files that are organized into the following folder structure,

    data
    ├── testing
    │   ├── 0
    │   ├── 1
    │   ├── 2
    │   ├── 3
    │   ├── 4
    │   ├── 5
    │   ├── 6
    │   ├── 7
    │   ├── 8
    │   └── 9
    └── training
        ├── 0
        ├── 1
        ├── 2
        ├── 3
        ├── 4
        ├── 5
        ├── 6
        ├── 7
        ├── 8
        └── 9

The subfolders under `data/training/` and `data/testing/` are the classification labels for the images in the dataset.
This folder structure is a standardized way to encode the label information and many libraries can make use of it, including the Tensorflow library that we are using for this particular demo.

## Submit and run jobs

The entrypoints that we will be running in this example are implemented in the Python source files under `src/` and the `MLproject` file.
To run these entrypoints within Dioptra's architecture, we need to package those files up into an archive and submit it to the Dioptra RESTful API to create a new job.
For convenience, the `Makefile` provides a rule for creating the archive file for this example, just run `make workflows`,

In [ ]:
%%bash

# Create the workflows.tar.gz file
make workflows

To connect with the endpoint, we will use a client class defined in the `utils.py` file that is able to connect with the Dioptra RESTful API using the HTTP protocol.
We connect using the client below, which uses the environment variable `DIOPTRA_RESTAPI_URI` to figure out how to connect to the Dioptra RESTful API,

In [ ]:
restapi_client = utils.DioptraClient(address=RESTAPI_API_BASE)

We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [ ]:
response_experiment = restapi_client.get_experiment_by_name(name=experiment)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=experiment)

response_experiment

We should also check which queues are available for running our jobs to make sure that the resources that we need are available.
The code below queries the Lab API and returns a list of active queues.

In [ ]:
### restapi_client.list_queues()  #Deprecated?

The rest of this section is divided into two parts.
Both parts investigate the effectiveness of the feature squeezing defense against several kinds of evasion attacks, but differ in the dataset and model architectures used.
The first part uses the MNIST dataset and LeNet-5 architecture, while the second part uses the ImageNet dataset and the mobilenet CNN architecture.

### MNIST data

We will be training the LeNet-5 model from scratch on the MNIST dataset,

In [ ]:
# Submit training job for the LeNet-5 network architecture
response_le_net_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="train",
    entry_point_kwargs=" ".join([
        f"-P register_model_name={mnist_model}",
        "-P model_architecture=le_net",
        "-P epochs=30",
        "-P data_dir=/nfs/data/Mnist"
    ]),
    queue = mlflow_queue,
)

print("Training job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_le_net_train)
#        "-P data_dir=/nfs/data/Mnist"
#  "-P register_model=True",

In [ ]:
# Submit training job for the shallownet network architecture
response_le_net_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="train",
    entry_point_kwargs=" ".join([
        f"-P register_model_name={mnist_shallow}",
        "-P model_architecture=shallow_net",
        "-P epochs=30",
        "-P data_dir=/nfs/data/Mnist"
    ]),
    queue = mlflow_queue,
)

print("Training job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_le_net_train)


The following code block generates adversarial images on the MNIST dataset using the Fast Gradient Method attack and then attempts to classify the adversarial images.

| parameter | type | description |
| --- | --- | --- |
| `eps` | _float_ | Attack step size. \[default: 0.3\] |
| `eps_step` | _float_ | Step size of input variation for minimal perturbation computation. [default: 0.1] |
| `targeted` | _bool_ | Indicates whether the attack is targeted (True) or untargeted (False). [default: False] |
| `num_random_init` | _int_ | Number of random initializations within the epsilon ball. For ``random_init=0`` starting at the original input. [default: 0] |
| `minimal` | _bool_ | Indicates if computing the minimal perturbation (True). If True, also define eps_step for the step size and eps for the maximum perturbation. [default: False] |

In [ ]:

def mlflow_run_id_is_not_known(response_fgm):
    return response_fgm["mlflowRunId"] is None and response_fgm["status"] not in [
        "failed",
        "finished",
    ]

response_fgm_le_net = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="fgm",
    entry_point_kwargs=" ".join(
        [f"-P model_name={mnist_model}","-P data_dir=/nfs/data/Mnist",f"-P model_version={model_id}"]
    ),
    queue = mlflow_queue,
    timeout="1m"

)

print("FGM attack job submitted")
print("")
pprint.pprint(response_fgm_le_net)
print("")

while mlflow_run_id_is_not_known(response_fgm_le_net):
    time.sleep(1)
    response_fgm_le_net = restapi_client.get_job_by_id(response_fgm_le_net["jobId"]) 
    
response_infer = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_fgm_le_net['mlflowRunId']}",
            f"-P model_version={model_id}",
            f"-P model_name={mnist_model}",
        ]
    ),
    queue = mlflow_queue,
    depends_on=response_fgm_le_net["jobId"]
)
    
print("Dependent jobs submitted")

This block does the same as the previous block, but applies the feature squeezing defense between the attack and infer steps.
This pre-processing defense compresses the images being classified by our neural network such that their color depth is reduced to a binary, monochrome pallete.
The level of compression can be tuned by adjusting the bit_depth parameter below (use values between 1 (binary) and 8 (original image color depth) to tune the defense.

**FGM parameters**

| parameter | type | description |
| --- | --- | --- |
| `eps` | _float_ | Attack step size. \[default: 0.3\] |
| `eps_step` | _float_ | Step size of input variation for minimal perturbation computation. [default: 0.1] |
| `targeted` | _bool_ | Indicates whether the attack is targeted (True) or untargeted (False). [default: False] |
| `num_random_init` | _int_ | Number of random initializations within the epsilon ball. For ``random_init=0`` starting at the original input. [default: 0] |
| `minimal` | _bool_ | Indicates if computing the minimal perturbation (True). If True, also define eps_step for the step size and eps for the maximum perturbation. [default: False] |

**Feature squeezing parameters**

| parameter | type | description |
| --- | --- | --- |
| `bit_depth` | _int_ | An integer between 1-8 that defines the color depth of the squeezed image. [default: 8] |

In [ ]:


def mlflow_run_id_is_not_known(response_fgm):
    return response_fgm["mlflowRunId"] is None and response_fgm["status"] not in [
        "failed",
        "finished",
    ]

response_fgm_le_net = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name= experiment,
    entry_point="fgm",
    entry_point_kwargs=" ".join(
        [f"-P model_name={mnist_model}","-P data_dir=/nfs/data/Mnist",f"-P model_version={model_id}"]

    ),
    queue = mlflow_queue,

)

print("FGM attack  job submitted")
print("")
pprint.pprint(response_fgm_le_net)
print("")

while mlflow_run_id_is_not_known(response_fgm_le_net):
    time.sleep(1)
    response_fgm_le_net = restapi_client.get_job_by_id(response_fgm_le_net["jobId"])

response_feature_squeeze = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="feature_squeeze",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_fgm_le_net['mlflowRunId']}",
            f"-P model={mnist_model}",
            "-P model_architecture=le_net",
            "-P bit_depth=1",
            "-P batch_size=32"
        ]
    ),
    queue = mlflow_queue,
    depends_on=response_fgm_le_net["jobId"],
    timeout = "1h"
)

while mlflow_run_id_is_not_known(response_feature_squeeze):
    time.sleep(1)
    response_feature_squeeze = restapi_client.get_job_by_id(response_feature_squeeze["jobId"]) 
    
response_infer_defended = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_feature_squeeze['mlflowRunId']}",
            f"-P model_version={model_id}",
            f"-P model_name={mnist_model}",
        ]
    ),
    queue = mlflow_queue,
    depends_on=response_feature_squeeze["jobId"]
)
    
print("Dependent jobs submitted")

This block uses the carlini-wagner attack with the Linf distance metric to generate adversarial images and checks the model's accuracy against the attack.

**Carlini Wagner Parameters**

| parameter | type | description |
| --- | --- | --- |
| `targeted` | _bool_ | Indicates whether the attack is targeted (True) or untargeted (False). [default: False] |
| `learning_rate` | _float_ | The initial learning rate for the attack algorithm. Smaller values produce better results but are slower to converge. [default: 0.01] |

In [ ]:
def mlflow_run_id_is_not_known(response_fgm):
    return response_fgm["mlflowRunId"] is None and response_fgm["status"] not in [
        "failed",
        "finished",
    ]

response_fgm_le_net = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="cw_inf",
    entry_point_kwargs=" ".join(
        [f"-P model_name={mnist_shallow}", f"-P model_version={model_id}", "-P model_architecture=shallow_net","-P targeted=True", "-P max_iter=10","-P confidence=0.0", "-P max_doubling=10","-P data_dir=/nfs/data/Mnist",  "-P learning_rate=0.01", "-P max_halving=10", "-P verbose=True", "-P batch_size=32"]

    ),
    queue = mlflow_queue,

)
#model_architecture=le_net
print("Carlini Wagner attack job submitted")
print("")
pprint.pprint(response_fgm_le_net)
print("")

while mlflow_run_id_is_not_known(response_fgm_le_net):
    time.sleep(1)
    response_fgm_le_net = restapi_client.get_job_by_id(response_fgm_le_net["jobId"])

response_le_net_infer_le_net_fgm = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="infer",
    entry_point_kwargs=" ".join(
         [
            f"-P run_id={response_fgm_le_net['mlflowRunId']}",
            f"-P model_version={model_id}",
            f"-P model_name={mnist_shallow}",
        ]
    ),
    queue = mlflow_queue,
    depends_on=response_fgm_le_net["jobId"],
)

print("Dependent jobs submitted")

This block does the same as the previous block, but applies the feature squeezing defense between the attack and infer steps.
This defense may be tuned and adjusted in the same manner as the previous feature squeeze block defending against the FGM attack.

**Carlini Wagner Parameters**

| parameter | type | description |
| --- | --- | --- |
| `targeted` | _bool_ | Indicates whether the attack is targeted (True) or untargeted (False). [default: False] |
| `learning_rate` | _float_ | The initial learning rate for the attack algorithm. Smaller values produce better results but are slower to converge. [default: 0.01] |

In [ ]:

def mlflow_run_id_is_not_known(response_fgm):
    return response_fgm["mlflowRunId"] is None and response_fgm["status"] not in [
        "failed",
        "finished",
    ]

response_fgm_le_net = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="cw_inf",
    entry_point_kwargs=" ".join(
        [f"-P model_name={mnist_shallow}", 
         f"-P model_version={model_id}", 
         "-P model_architecture=shallow_net",
         "-P targeted=True", 
         "-P max_iter=20",
         "-P confidence=0.0", 
         "-P max_doubling=10",
         "-P data_dir=/nfs/data/Mnist", 
         "-P learning_rate=0.01", 
         "-P max_halving=10", 
         "-P verbose=True", 
         "-P batch_size=32"]

    ),
    queue = mlflow_queue,
)

print("Carlini Wagner attack job submitted")
print("")
pprint.pprint(response_fgm_le_net)
print("")

while mlflow_run_id_is_not_known(response_fgm_le_net):
    time.sleep(1)
    response_fgm_le_net = restapi_client.get_job_by_id(response_fgm_le_net["jobId"])

response_feature_squeeze = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="feature_squeeze",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_fgm_le_net['mlflowRunId']}",
            f"-P model_name={mnist_shallow}",
            f"-P model_version={model_id}"
            "-P model_architecture=shallow_net",
            "-P bit_depth=1",
            "-P batch_size=32"
        ]
    ),
    depends_on=response_fgm_le_net["jobId"],
)

while mlflow_run_id_is_not_known(response_feature_squeeze):
    time.sleep(1)
    response_feature_squeeze = restapi_client.get_job_by_id(response_fgm_le_net["jobId"])

response_le_net_infer_le_net_fgm = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_feature_squeeze['mlflowRunId']}",
            f"-P model_name={mnist_shallow}",
            f"-P model_version={model_id}"
            "-P model_architecture=shallow_net",
        ]
    ),
    depends_on=response_feature_squeeze["jobId"],
)

print("Dependent jobs submitted")

This block uses the carlini-wagner attack using the L2 distance metric to generate adversarial images and checks the model's accuracy against the attack.

In [ ]:


def mlflow_run_id_is_not_known(response_fgm):
    return response_fgm["mlflowRunId"] is None and response_fgm["status"] not in [
        "failed",
        "finished",
    ]

response_fgm_le_net = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="cw_l2",
    entry_point_kwargs=" ".join(
        [f"-P model_name={mnist_shallow}",f"-P model_version={model_id}", "-P binary_search_steps=50", "-P initial_const=0.01", "-P model_architecture=shallow_net", "-P max_iter=10", "-P max_doubling=5","-P data_dir=/nfs/data/Mnist", "-P max_halving=5", "-P verbose=True", "-P batch_size=32"]

    ),
    queue = mlflow_queue,

)

print("Carlini Wagner attack job submitted")
print("")
pprint.pprint(response_fgm_le_net)
print("")

while mlflow_run_id_is_not_known(response_fgm_le_net):
    time.sleep(1)
    response_fgm_le_net = restapi_client.get_job_by_id(response_fgm_le_net["jobId"])

response_le_net_infer_le_net_fgm = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_fgm_le_net['mlflowRunId']}",
            f"-P model_name={mnist_model}",
            f"-P model_version={model_id}",
            "-P model_architecture=shallow_net",
            "-P batch_size=32"
        ]
    ),
    queue = mlflow_queue,

    depends_on=response_fgm_le_net["jobId"],
)

print("Dependent jobs submitted")


This block does the same as the previous block, but applies the feature squeezing defense between the attack and infer steps.

In [ ]:
def mlflow_run_id_is_not_known(response_fgm):
    return response_fgm["mlflowRunId"] is None and response_fgm["status"] not in [
        "failed",
        "finished",
    ]

response_fgm_le_net = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="cw",
    entry_point_kwargs=" ".join(
        [
            f"-P model={mnist_model}",
            "-P binary_search_steps=50",
            "-P initial_const=0.01",
            "-P model_architecture=le_net",
            "-P max_iter=10",
            "-P max_doubling=5",
def mlflow_run_id_is_not_known(response_fgm):
    return response_fgm["mlflowRunId"] is None and response_fgm["status"] not in [
        "failed",
        "finished",
    ]

response_fgm_le_net = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="cw_l2",
    entry_point_kwargs=" ".join(
        [f"-P model_name={mnist_shallow}",f"-P model_version={model_id}","-P binary_search_steps=50", "-P initial_const=0.01", "-P model_architecture=shallow_net", "-P max_iter=10", "-P max_doubling=5","-P data_dir=/nfs/data/Mnist", "-P max_halving=5", "-P verbose=True", "-P batch_size=32"]

    ),
    queue = mlflow_queue,
)

print("Carlini Wagner attack  job submitted")
print("")
pprint.pprint(response_fgm_le_net)
print("")

while mlflow_run_id_is_not_known(response_fgm_le_net):
    time.sleep(1)
    response_fgm_le_net = restapi_client.get_job_by_id(response_fgm_le_net["jobId"])

response_feature_squeeze = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="feature_squeeze",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_fgm_le_net['mlflowRunId']}",
            f"-P model_name={mnist_shallow}",
            f"-P model_version{model_id}",
            "-P model_architecture=shallow_net",
            "-P bit_depth=1",
            "-P batch_size=32"
        ]
    ),
    depends_on=response_fgm_le_net["jobId"],
    queue = mlflow_queue,
)

while mlflow_run_id_is_not_known(response_feature_squeeze):
    time.sleep(1)
    response_feature_squeeze = restapi_client.get_job_by_id(response_feature_squeeze["jobId"])

response_le_net_infer_le_net_fgm = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=expperiment,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_feature_squeeze['mlflowRunId']}",
            f"-P model_name={mnist_model}",
            f"-P model_version={model_id}"
            "-P model_architecture=shallow_net",
            "-P batch_size=32"
        ]
    ),
    depends_on=response_feature_squeeze["jobId"],
    queue = mlflow_queue,
)

print("Dependent jobs submitted")
            f"-P data_dir={data_path_mnist}",
            "-P max_halving=5",
            "-P verbose=True",
            "-P batch_size=32",
        ],
    ),
    queue = mlflow_queue,
)

print("Carlini Wagner attack (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_fgm_le_net)
print("")

while mlflow_run_id_is_not_known(response_fgm_le_net):
    time.sleep(1)
    response_fgm_le_net = restapi_client.get_job_by_id(response_fgm_le_net["jobId"])

response_feature_squeeze = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="feature_squeeze",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_fgm_le_net['mlflowRunId']}",
            f"-P model={mnist_shallow}",
            "-P model_architecture=shallow_net",
            "-P bit_depth=1",
            "-P batch_size=32"
        ],
    ),
    depends_on=response_fgm_le_net["jobId"],
    queue = mlflow_queue,
)

while mlflow_run_id_is_not_known(response_feature_squeeze):
    time.sleep(1)
    response_feature_squeeze = restapi_client.get_job_by_id(response_feature_squeeze["jobId"])

response_le_net_infer_le_net_fgm = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=expperiment,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_feature_squeeze['mlflowRunId']}",
            f"-P model={mnist_shallow}",
            "-P model_architecture=shallow_net",
            "-P batch_size=32"
        ],
    ),
    depends_on=response_feature_squeeze["jobId"],
    queue = mlflow_queue,
)

print("Dependent jobs submitted")

This block uses the Deepfool attack to generate adversarial MNIST images, applies the feature squeezing defense, and checks the model's accuracy against the defended adversarial dataset.

**Unique Deepfool parameters**

| parameter | type | description |
| --- | --- | --- |
| `epsilon` | _float_ | Overshoot parameter. [default: 0.00001] |
| `nb_grads` | _int_ | Number of class gradients to compute. [default: 10] |

In [ ]:
def mlflow_run_id_is_not_known(response_fgm):
    return response_fgm["mlflowRunId"] is None and response_fgm["status"] not in [
        "failed",
        "finished",
    ]

response_fgm_le_net = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="deepfool",
    entry_point_kwargs=" ".join(
        [
            f"-P model={mnist_model}",
            f"-P data_dir={data_path_mnist}/training",
            "-P model_architecture=le_net",
            "-P batch_size=32",
            "-P max_iter=10",
            "-P verbose=True",
            "-P nb_grads=10",
            "-P epsilon=0.000001",
        ],
    ),
    queue = mlflow_queue,
)

print("Deepfool attack (Mobilenet architecture) job submitted")
print("")
pprint.pprint(response_fgm_le_net)
print("")

while mlflow_run_id_is_not_known(response_fgm_le_net):
    time.sleep(1)
    response_fgm_le_net = restapi_client.get_job_by_id(response_fgm_le_net["jobId"])

response_feature_squeeze = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="feature_squeeze",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_fgm_le_net['mlflowRunId']}",
            f"-P model={mnist_model}",
            "-P model_architecture=le_net",
            "-P bit_depth=1",
            "-P batch_size=32",
            f"-P data_dir={data_path_mnist}/training",
        ],
    ),
    depends_on=response_fgm_le_net["jobId"],
    queue = mlflow_queue,
)

while mlflow_run_id_is_not_known(response_feature_squeeze):
    time.sleep(1)
    response_feature_squeeze = restapi_client.get_job_by_id(response_feature_squeeze["jobId"])

response_le_net_infer_le_net_fgm = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_feature_squeeze['mlflowRunId']}",
            f"-P model={mnist_model}",
            "-P model_architecture=le_net",
            "-P batch_size=32"
        ],
    ),
    depends_on=response_feature_squeeze["jobId"],
    queue = mlflow_queue,
)

print("Dependent jobs submitted")

This block uses the Deepfool attack to generate adversarial MNIST images and checks the model's accuracy against the attack.

**Unique Deepfool parameters**

| parameter | type | description |
| --- | --- | --- |
| `epsilon` | _float_ | Overshoot parameter. [default: 0.00001] |
| `nb_grads` | _int_ | Number of class gradients to compute. [default: 10] |

In [ ]:
def mlflow_run_id_is_not_known(response_fgm):
    return response_fgm["mlflowRunId"] is None and response_fgm["status"] not in [
        "failed",
        "finished",
    ]

response_fgm_le_net = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="deepfool",
    entry_point_kwargs=" ".join(
        [
            f"-P model={mnist_model}",
            f"-P data_dir={data_path_mnist}/training",
            "-P model_architecture=le_net",
            "-P batch_size=32",
            "-P max_iter=10",
            "-P verbose=True",
            "-P nb_grads=10",
            "-P epsilon=0.000001",
        ],
    ),
    queue = mlflow_queue,
)

print("Deepfool attack (Mobilenet architecture) job submitted")
print("")
pprint.pprint(response_fgm_le_net)
print("")

while mlflow_run_id_is_not_known(response_fgm_le_net):
    time.sleep(1)
    response_fgm_le_net = restapi_client.get_job_by_id(response_fgm_le_net["jobId"])

response_le_net_infer_le_net_fgm = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=example,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_fgm_le_net['mlflowRunId']}",
            f"-P model={mnist_model}",
            "-P model_architecture=le_net",
            "-P batch_size=32"
        ]
    ),
    depends_on=response_fgm_le_net["jobId"],
    queue = mlflow_queue,
)

print("Dependent jobs submitted")

This block applies the Jacobian Saliency Map Approach attack to generate adversarial images for the MNIST dataset.

**Unique JSMA parameters**

| parameter | type | description |
| --- | --- | --- |
| `theta` | _float_ | Amount of Perturbation introduced to each modified feature per step (can be positive or negative). [default: 0.1] |
| `gamma` | _float_ | Maximum fraction of features being perturbed (between 0 and 1). [default: 1.0] |

In [ ]:

def mlflow_run_id_is_not_known(response_fgm):
    return response_fgm["mlflowRunId"] is None and response_fgm["status"] not in [
        "failed",
        "finished",
    ]

response_jsma = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="jsma",
    entry_point_kwargs=" ".join(
        [f"-P model_name={mnist_model}",
         f"-P model_version={model_id}", 
         "-P model_architecture=le_net", 
         "-P theta=4.5",
         "-P verbose=True",
         "-P gamma=1.0", 
         "-P data_dir=/nfs/data/Mnist"] 

    ),
    queue = mlflow_queue,

)

print("JSMA attack (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_jsma)
print("")


while mlflow_run_id_is_not_known(response_jsma):
    time.sleep(1)
    response_jsma = restapi_client.get_job_by_id(response_jsma["jobId"])

response_le_net_infer_le_net_fgm = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_jsma['mlflowRunId']}",
            f"-P model_name={mnist_model}",
            f"-P model_version={model_id}",
        ]
    ),
    queue = mlflow_queue,
    depends_on=response_jsma["jobId"],
)

print("Dependent jobs submitted")

This block does the same as the previous block, but applies the feature squeezing defense between the attack and infer steps.

In [ ]:
def mlflow_run_id_is_not_known(response_fgm):
    return response_fgm["mlflowRunId"] is None and response_fgm["status"] not in [
        "failed",
        "finished",
    ]

response_jsma = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="jsma",
    entry_point_kwargs=" ".join(
        [f"-P model_name={mnist_model}",
         f"-P model_version={model_id}", 
         "-P model_architecture=le_net", 
         "-P theta=4.5",
         "-P verbose=True",
         "-P gamma=1.0", 
         "-P data_dir=/nfs/data/Mnist"] 
    ),
    queue = mlflow_queue,
)

print("JSMA attack (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_jsma)
print("")

while mlflow_run_id_is_not_known(response_jsma):
    time.sleep(1)
    response_jsma = restapi_client.get_job_by_id(response_jsma["jobId"])

response_feature_squeeze = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="feature_squeeze",
    entry_point_kwargs=" ".join(
           [
            f"-P run_id={response_jsma['mlflowRunId']}",
            f"-P model_name={mnist_model}",
            f"-P model_version={model_id}"
            "-P model_architecture=le_net",
            "-P bit_depth=1",
            "-P batch_size=32"
        ]
    ),
    queue = mlflow_queue,
    depends_on=response_jsma["jobId"],
)

while mlflow_run_id_is_not_known(response_feature_squeeze):
    time.sleep(1)
    response_feature_squeeze = restapi_client.get_job_by_id(response_feature_squeeze["jobId"])

response_le_net_infer_le_net_jsma = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_jsma['mlflowRunId']}",
            f"-P model_name={mnist_model}",
            f"-P model_version={model_id}",
        ]
    ),
    queue = mlflow_queue,
    depends_on=response_feature_squeeze["jobId"],
)

print("Dependent jobs submitted")

### ImageNet data

This block loads a pretrained mobilenet model from `tensorflow.applications`.
This is required for sections of the demo classifying images from the Imagenet dataset, specifically those involving the Deepfool attack.

In [ ]:

# Submit training job for the LeNet-5 network architecture
response_le_net_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="init_model",
    entry_point_kwargs=" ".join([
        "-P model_architecture=mobilenet",   
        "-P data_dir=/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-10000",
        "-P seed=-1",
        "-P batch_size=40"
    ]),
    queue = mlflow_queue,
)
# mobilenet",
print("Training job for mobilenet neural network submitted")
print("")
pprint.pprint(response_le_net_train)


This block uses the Deepfool attack to generate adversarial images and checks the model's accuracy against the attack.
Please note that this attack uses the mobilenet CNN architecture and generates adversarial images of the ImageNet dataset.
This attack may take quite some time to complete.
Due to the long compute times, you may want to run this attack in stages.
In a lower cell, we've provided a method for loading an adversarial dataset and running the later steps of the pipeline via runid.

**Unique Deepfool parameters**

| parameter | type | description |
| --- | --- | --- |
| `epsilon` | _float_ | Overshoot parameter. [default: 0.00001] |
| `nb_grads` | _int_ | Number of class gradients to compute. [default: 10] |

In [ ]:
def mlflow_run_id_is_not_known(response_df): #Redundant function, but lets us skip FGM cells if desired
    return response_df["mlflowRunId"] is None and response_df["status"] not in [
        "failed",
        "finished",
    ]

response_df_le_net = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="deepfool",
    entry_point_kwargs=" ".join(
        [
            f"-P model={imagenet_model}",
            f"-P data_dir={data_path_imagenet}",
            "-P model_architecture=mobilenet",
            "-P batch_size=40",
            "-P max_iter=10",
            "-P verbose=True",
            "-P nb_grads=10",
            "-P epsilon=0.000001",
        ],
    ),
    queue = mlflow_queue,
)

print("DeepFool attack (Mobilenet architecture) job submitted")
print("")
pprint.pprint(response_df_le_net)
print("")

while mlflow_run_id_is_not_known(response_df_le_net):
    time.sleep(1)
    response_df_le_net = restapi_client.get_job_by_id(response_df_le_net["jobId"])

response_le_net_infer_le_net_df = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_df_le_net['mlflowRunId']}",
            f"-P model={imagenet_model}",
            "-P model_architecture=mobilenet",
            "-P batch_size=40"
        ]
    ),
    queue = mlflow_queue,
    depends_on=response_df_le_net["jobId"],
)

print("Dependent jobs submitted")

This block does the same as the initial Deepfool block, but adds in a step where the feature squeezing defense is applied before the infer step.
As in previous defended code blocks, this may be tuned by adjusting the bit-depth parameter.
Note that imagenet uses RGB images, and feature squeezing is applied equally on all channels (meaning that `bit_depth=4` corresponds to R = 4, B = 4, G = 4). 

**Unique Deepfool parameters**

| parameter | type | description |
| --- | --- | --- |
| `epsilon` | _float_ | Overshoot parameter. [default: 0.00001] |
| `nb_grads` | _int_ | Number of class gradients to compute. [default: 10] |

In [ ]:
def mlflow_run_id_is_not_known(response_fgm):
    return response_fgm["mlflowRunId"] is None and response_fgm["status"] not in [
        "failed",
        "finished",
    ]

response_df_mobilenet = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="deepfool",
    entry_point_kwargs=" ".join(
        [
            f"-P model={imagenet_model}",
            f"-P data_dir={data_path_imagenet}",
            "-P model_architecture=mobilenet",
            "-P batch_size=40",
            "-P max_iter=10",
            "-P verbose=True",
            "-P nb_grads=10",
            "-P epsilon=1",
        ],
    ),
    queue = mlflow_queue,
)

print("Deepfool attack (Mobilenet architecture) job submitted")
print("")
pprint.pprint(response_df_mobilenet)
print("")

while mlflow_run_id_is_not_known(response_df_mobilenet):
    time.sleep(1)
    response_df_mobilenet = restapi_client.get_job_by_id(response_df_mobilenet["jobId"])

response_feature_squeeze = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="feature_squeeze",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_df_mobilenet['mlflowRunId']}",
            f"-P model={imagenet_model}",
            "-P model_architecture=mobilenet",
            "-P bit_depth=1",
            "-P batch_size=40"
        ]
    ),
    depends_on=response_df_mobilenet["jobId"],
    queue = mlflow_queue,
)

while mlflow_run_id_is_not_known(response_feature_squeeze):
    time.sleep(1)
    response_feature_squeeze = restapi_client.get_job_by_id(response_feature_squeeze["jobId"])

response_le_net_infer_le_net_df = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_feature_squeeze['mlflowRunId']}",
            f"-P model={imagenet_model}",
            "-P model_architecture=mobilenet",
            "-P batch_size=40"
        ]
    ),
    depends_on=response_feature_squeeze["jobId"],
    queue = mlflow_queue,
)

print("Dependent jobs submitted")

This block can be used to manually run both the squeeze and infer steps on a previous deepfool run. 
Use this to avoid the adversarial image generation step, which may be time consuming.
To run this block, you must specify a valid `run_id` and `model`.
To manually fetch a run ID, you can navigate to MLFlow Dashboard and select a successful deepfool run.
The run ID will be listed on the page header.

In [ ]:
def mlflow_run_id_is_not_known(response_fgm):
    return response_fgm["mlflowRunId"] is None and response_fgm["status"] not in [
        "failed",
        "finished",
    ]


runid = "5cb28628c51b4541bcb68276525f7f1d"

response_feature_squeeze = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="feature_squeeze",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={runid}",
            f"-P model={imagenet_model}",
            "-P model_architecture=mobilenet",
            "-P bit_depth=4",
            "-P batch_size=40",
        ],
    ),
    queue = mlflow_queue,
)

while mlflow_run_id_is_not_known(response_feature_squeeze):
    time.sleep(1)
    response_feature_squeeze = restapi_client.get_job_by_id(response_feature_squeeze["jobId"])

response_le_net_infer_le_net_df = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=experiment,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_feature_squeeze['mlflowRunId']}",
            f"-P model={imagenet_model}",
            "-P model_architecture=mobilenet",
            "-P batch_size=40",
        ],
    ),
    depends_on=response_feature_squeeze["jobId"],
    queue = mlflow_queue,
)

print("Dependent jobs submitted")